In [ ]:
# Code to read csv file into Colaboratory:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/"

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation, Dense
from datetime import datetime

!pip install -U tensorboard_plugin_profile
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

In [ ]:
df = pd.read_csv('full_2019_2grid.csv')
#df = df.iloc[:500000]

In [ ]:
# Confirm tensorflow can access GPU
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Train 
df_30 = df.drop(['y_60', 'timestamp'], axis=1)
df_30['grid_num'] = df_30['grid_num'].astype(str)

In [ ]:
# split into train test
df_30 = pd.get_dummies(df_30, columns = ['grid_num','month', 'hour'])

X = df_30.iloc[:,df_30.columns != 'y_30']
y = df_30['y_30']

#get the stratified train_test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = 622
)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
model = Sequential()
model.add(Dense(1, activation = 'linear'))

model.compile(optimizer='adam',loss='mse')

# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

train = train_dataset.batch(128)
test = test_dataset.batch(128)
model.fit(train,
          epochs=2,
          validation_data=test,
          callbacks = [tboard_callback])

#model.fit(x=X_train,y=y_train.values,
#          validation_data=(X_test,y_test.values),
#          batch_size=128,epochs=10,
#          callbacks = [tboard_callback])

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs

In [ ]:
ds_train = train_dataset.batch(128)
ds_train = ds_train.cache()
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = test_dataset.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model.fit(ds_train,
          epochs=2,
          validation_data=ds_test,
          callbacks = [tboard_callback])

In [ ]:
%tensorboard --logdir=logs

In [ ]:
model.evaluate(ds_test)